## Model Train 1 - Tesis Javier-Uriel

### Importamos algunas librerías que nos serán útiles más adelante

In [41]:
import os
import json
import random
import tensorflow as tf
assert (tf.__version__=='2.3.0'), 'Versión incorrecta de Tensorflow, por favor instale 2.3.0'

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

pd.set_option('display.max_columns', None) #Para mostrar todas las columnas

### Leemos el Dataset

In [11]:
#Dataset solo movimientos en Z
rpm_list = ['RPM0', 'RPM1', 'RPM2', 'RPM3']
states_list_org = ["vz","az", "uvz", 
                   "p", "q",
                   "wp", "wq", 
                   "ap", "aq"]
states_list_min = ["vz","az", "uvz"]
dataset_name = "Dataset_Z_Disturbance"
directory = "../logs/Datasets/"+dataset_name
ORDER = 3
dfs = []
states_list=states_list_org.copy()

In [12]:
for filename in os.listdir(directory):
    if not filename.endswith(".csv"):
        continue
    df = pd.read_csv(os.path.join(directory, filename))
    a = []
    ## Desplazamos estados anteriores        
    for column in states_list:
        for n in range(1,ORDER+1):
            df[column+str(n)] = df[column].shift(periods=n, fill_value=0)
            a.append(column+str(n))
    dfs.append(df)
states_list+=a        
        
dataset = pd.concat(dfs)
dataset.head()

,timestamps,x,y,z,Q1,Q2,Q3,Q4,p,q,r,vx,vy,vz,wp,wq,wr,ax,ay,az,ap,aq,ar,RPM0,RPM1,RPM2,RPM3,ux,uy,uz,uvx,uvy,uvz,up,uq,ur,uwp,uwq,uwr,vz1,vz2,vz3,az1,az2,az3,uvz1,uvz2,uvz3,p1,p2,p3,q1,q2,q3,wp1,wp2,wp3,wq1,wq2,wq3,ap1,ap2,ap3,aq1,aq2,aq3
0,0.000000,0.0,0.0,24.999830,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.040833,0.0,0.0,0.0,0.0,0.0,-9.800000,0.0,0.0,0.0,15016.253501,15016.253501,15016.253501,15016.253501,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.004167,0.0,0.0,24.999673,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.037676,0.0,0.0,0.0,0.0,0.0,0.757873,0.0,0.0,0.0,15016.253501,15016.253501,15016.253501,15016.253501,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.040833,0.000000,0.000000,-9.800000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.008333,0.0,0.0,24.999529,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.034518,0.0,0.0,0.0,0.0,0.0,0.757737,0.0,0.0,0.0,15016.253501,15016.253501,15016.253501,15016.253501,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.037676,-0.040833,0.000000,0.757873,-9.800000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.012500,0.0,0.0,24.999398,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.031362,0.0,0.0,0.0,0.0,0.0,0.757601,0.0,0.0,0.0,15016.253501,15016.253501,15016.253501,15016.253501,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.034518,-0.037676,-0.040833,0.757737,0.757873,-9.800000,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.016667,0.0,0.0,24.999281,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.028206,0.0,0.0,0.0,0.0,0.0,0.757467,0.0,0.0,0.0,15016.253501,15016.253501,15016.253501,15016.253501,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.031362,-0.034518,-0.037676,0.757601,0.757737,0.757873,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Estados repetidos

En este caso se eliminan estados repetidos y estados que se encuentren en estado transitorio mientras el dron despega o se estabiliza antes de introducir la señal de control.

In [13]:
dataset_1 = dataset.drop(["timestamps"], axis=1)
print(f'shape (b4 drop) = {dataset_1.shape}')
dataset_dropped = dataset_1.drop_duplicates()
print(f'shape = {dataset_dropped.shape}')
print(f'len (b4 drop) - len = {dataset.shape[0]-dataset_dropped.shape[0]}')
print(f'columns = {dataset_dropped.columns}')

shape (b4 drop) = (1439940, 65)
shape = (1439720, 65)
len (b4 drop) - len = 220
columns = Index(['x', 'y', 'z', 'Q1', 'Q2', 'Q3', 'Q4', 'p', 'q', 'r', 'vx', 'vy', 'vz',
       'wp', 'wq', 'wr', 'ax', 'ay', 'az', 'ap', 'aq', 'ar', 'RPM0', 'RPM1',
       'RPM2', 'RPM3', 'ux', 'uy', 'uz', 'uvx', 'uvy', 'uvz', 'up', 'uq', 'ur',
       'uwp', 'uwq', 'uwr', 'vz1', 'vz2', 'vz3', 'az1', 'az2', 'az3', 'uvz1',
       'uvz2', 'uvz3', 'p1', 'p2', 'p3', 'q1', 'q2', 'q3', 'wp1', 'wp2', 'wp3',
       'wq1', 'wq2', 'wq3', 'ap1', 'ap2', 'ap3', 'aq1', 'aq2', 'aq3'],
      dtype='object')


In [14]:
states = dataset_dropped[states_list]
print(f'columns = {states.columns}')
print(f'shape = {states.shape}')
states.head()

columns = Index(['vz', 'az', 'uvz', 'p', 'q', 'wp', 'wq', 'ap', 'aq', 'vz1', 'vz2',
       'vz3', 'az1', 'az2', 'az3', 'uvz1', 'uvz2', 'uvz3', 'p1', 'p2', 'p3',
       'q1', 'q2', 'q3', 'wp1', 'wp2', 'wp3', 'wq1', 'wq2', 'wq3', 'ap1',
       'ap2', 'ap3', 'aq1', 'aq2', 'aq3'],
      dtype='object')
shape = (1439720, 36)


,vz,az,uvz,p,q,wp,wq,ap,aq,vz1,vz2,vz3,az1,az2,az3,uvz1,uvz2,uvz3,p1,p2,p3,q1,q2,q3,wp1,wp2,wp3,wq1,wq2,wq3,ap1,ap2,ap3,aq1,aq2,aq3
0,-0.040833,-9.800000,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.037676,0.757873,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,-0.040833,0.000000,0.000000,-9.800000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.034518,0.757737,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,-0.037676,-0.040833,0.000000,0.757873,-9.800000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.031362,0.757601,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,-0.034518,-0.037676,-0.040833,0.757737,0.757873,-9.800000,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.028206,0.757467,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,-0.031362,-0.034518,-0.037676,0.757601,0.757737,0.757873,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### División del dataset en estados y acciones

In [15]:
rpm_list = ['RPM0', 'RPM1', 'RPM2', 'RPM3']
actions = dataset[rpm_list]
print(f'columns = {actions.columns}')
print(f'shape = {actions.shape}')
actions.head()

columns = Index(['RPM0', 'RPM1', 'RPM2', 'RPM3'], dtype='object')
shape = (1439940, 4)


,RPM0,RPM1,RPM2,RPM3
0,15016.253501,15016.253501,15016.253501,15016.253501
1,15016.253501,15016.253501,15016.253501,15016.253501
2,15016.253501,15016.253501,15016.253501,15016.253501
3,15016.253501,15016.253501,15016.253501,15016.253501
4,15016.253501,15016.253501,15016.253501,15016.253501


#### Normalización de acciones

In [18]:
# def normalize_df(df, properties, type_norm="Z-Score"):
#     df_norm = pd.DataFrame()
#     dict_k = {}
#     for prop in properties:
#         dict_k[prop] = {}
#         dict_k[prop]["mean"] = df[prop].mean()
#         dict_k[prop]["max"] = df[prop].max()
#         dict_k[prop]["min"] = df[prop].min()
#         dict_k[prop]["std"] = df[prop].std()
#         if type_norm=="Z-Score":
#             df_norm[prop] = (df[prop]-dict_k[prop]["mean"])/dict_k[prop]["std"]
#         elif type_norm=="Log":
#             df_norm[prop] = np.log10(df[prop])
#         elif type_norm=="Linear_mean":
#             df_norm[prop] = (df[prop]-dict_k[prop]["min"])/(dict_k[prop]["max"]-dict_k[prop]["min"])
#         else: #Lineal mean
#             df_norm[prop] = (df[prop]-dict_k[prop]["mean"])/(dict_k[prop]["max"]-dict_k[prop]["min"])
#     dict_k["Norm_type"]=type_norm
#     return df_norm, dict_k

def normalize_df(df):
    K = df.max().max()
    df_norm = actions/K
    return df_norm, {'K':K}

In [21]:
actions_norm, K = normalize_df(actions)
print(K)
f = open(f"../logs/Files/dict_act_{dataset_name}.json","w")
f.write(json.dumps(K))
f.close()
actions_norm.describe()

{'K': 21666.4475}


,RPM0,RPM1,RPM2,RPM3
count,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06
mean,6.666977e-01,6.666939e-01,6.666968e-01,6.666952e-01
std,4.373702e-02,4.379628e-02,4.375531e-02,4.378075e-02
min,4.357106e-01,4.357106e-01,4.357106e-01,4.357106e-01
25%,6.615808e-01,6.615104e-01,6.615855e-01,6.615104e-01
50%,6.677809e-01,6.677808e-01,6.677809e-01,6.677806e-01
75%,6.749159e-01,6.749839e-01,6.749401e-01,6.749693e-01
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


#### Definimos los estados

In [23]:
states = dataset[states_list]
print(f'columns = {states.columns}')
print(f'shape = {states.shape}')
states.describe()

columns = Index(['vz', 'az', 'uvz', 'p', 'q', 'wp', 'wq', 'ap', 'aq', 'vz1', 'vz2',
       'vz3', 'az1', 'az2', 'az3', 'uvz1', 'uvz2', 'uvz3', 'p1', 'p2', 'p3',
       'q1', 'q2', 'q3', 'wp1', 'wp2', 'wp3', 'wq1', 'wq2', 'wq3', 'ap1',
       'ap2', 'ap3', 'aq1', 'aq2', 'aq3'],
      dtype='object')
shape = (1439940, 36)


,vz,az,uvz,p,q,wp,wq,ap,aq,vz1,vz2,vz3,az1,az2,az3,uvz1,uvz2,uvz3,p1,p2,p3,q1,q2,q3,wp1,wp2,wp3,wq1,wq2,wq3,ap1,ap2,ap3,aq1,aq2,aq3
count,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06,1.439940e+06
mean,6.552402e-02,1.861050e-03,6.295316e-02,3.050909e-05,1.759260e-06,1.735212e-06,1.165926e-06,6.231257e-19,-2.427330e-19,6.551627e-02,6.550848e-02,6.550065e-02,1.869911e-03,1.878779e-03,1.887653e-03,6.296014e-02,6.296711e-02,6.297409e-02,3.050909e-05,3.050909e-05,3.050909e-05,1.759260e-06,1.759260e-06,1.759260e-06,1.735212e-06,1.735212e-06,1.735212e-06,1.165926e-06,1.165926e-06,1.165926e-06,4.903406e-19,6.588540e-19,5.755926e-19,-2.437220e-19,-4.389830e-19,-6.199034e-19
std,7.761390e-01,1.282651e+00,7.866672e-01,1.135355e-02,1.140876e-02,7.966545e-02,8.097472e-02,1.296093e+00,1.329560e+00,7.761254e-01,7.761118e-01,7.760981e-01,1.282613e+00,1.282574e+00,1.282535e+00,7.866488e-01,7.866304e-01,7.866119e-01,1.135355e-02,1.135355e-02,1.135355e-02,1.140876e-02,1.140876e-02,1.140876e-02,7.966545e-02,7.966545e-02,7.966545e-02,8.097472e-02,8.097472e-02,8.097472e-02,1.296093e+00,1.296093e+00,1.296093e+00,1.329560e+00,1.329560e+00,1.329560e+00
min,-3.383819e+00,-9.800000e+00,-3.415860e+00,-3.159702e-01,-1.984465e-01,-2.426040e+00,-1.997995e+00,-1.726701e+01,-1.591711e+01,-3.383819e+00,-3.383819e+00,-3.383819e+00,-9.800000e+00,-9.800000e+00,-9.800000e+00,-3.415860e+00,-3.415860e+00,-3.415860e+00,-3.159702e-01,-3.159702e-01,-3.159702e-01,-1.984465e-01,-1.984465e-01,-1.984465e-01,-2.426040e+00,-2.426040e+00,-2.426040e+00,-1.997995e+00,-1.997995e+00,-1.997995e+00,-1.726701e+01,-1.726701e+01,-1.726701e+01,-1.591711e+01,-1.591711e+01,-1.591711e+01
25%,-2.021931e-01,-1.403078e-01,-2.068478e-01,-8.201255e-17,-1.472451e-16,-1.309726e-15,-1.488381e-15,-8.013511e-14,-8.013475e-14,-2.021708e-01,-2.021456e-01,-2.021286e-01,-1.402777e-01,-1.402441e-01,-1.402100e-01,-2.068288e-01,-2.067875e-01,-2.067454e-01,-8.201255e-17,-8.201255e-17,-8.201255e-17,-1.472372e-16,-1.472330e-16,-1.472090e-16,-1.309726e-15,-1.309726e-15,-1.309726e-15,-1.488226e-15,-1.488127e-15,-1.488111e-15,-8.013511e-14,-8.013511e-14,-8.013511e-14,-8.013475e-14,-8.013475e-14,-8.013475e-14
50%,1.967083e-10,0.000000e+00,0.000000e+00,1.222482e-19,-0.000000e+00,-1.595121e-21,9.772202e-21,-2.366583e-29,0.000000e+00,1.910388e-10,1.859235e-10,1.803089e-10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.210414e-19,1.201565e-19,1.187456e-19,-0.000000e+00,-0.000000e+00,0.000000e+00,-1.050439e-21,-7.892058e-22,-5.513017e-22,6.240923e-21,5.111430e-21,4.328449e-21,-2.366583e-29,-2.366583e-29,-2.366583e-29,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.176553e-01,1.452529e-01,3.171027e-01,1.245103e-16,1.566687e-16,1.268140e-15,1.546857e-15,8.013232e-14,8.013473e-14,3.176223e-01,3.175571e-01,3.174873e-01,1.452296e-01,1.452038e-01,1.451872e-01,3.171027e-01,3.171027e-01,3.171027e-01,1.245103e-16,1.245103e-16,1.245103e-16,1.566687e-16,1.566687e-16,1.566687e-16,1.268140e-15,1.268140e-15,1.268140e-15,1.546857e-15,1.546857e-15,1.546857e-15,8.013230e-14,8.013230e-14,8.013230e-14,8.013473e-14,8.013473e-14,8.013473e-14
max,3.649215e+00,1.276987e+01,3.685862e+00,2.378609e-01,1.985381e-01,2.533457e+00,1.873180e+00,1.576246e+01,1.610368e+01,3.649215e+00,3.649215e+00,3.649215e+00,1.276987e+01,1.276987e+01,1.276987e+01,3.685862e+00,3.685862e+00,3.685862e+00,2.378609e-01,2.378609e-01,2.378609e-01,1.985381e-01,1.985381e-01,1.985381e-01,2.533457e+00,2.533457e+00,2.533457e+00,1.873180e+00,1.873180e+00,1.873180e+00,1.576246e+01,1.576246e+01,1.576246e+01,1.610368e+01,1.610368e+01,1.610368e+01


### Dividimos el dataset

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(states, actions_norm, test_size=0.2)

## Keras Model

#### Early Stopping

In [25]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

#### Definición del Modelo

In [26]:
inputs = tf.keras.Input(shape=(states.shape[1],))
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(actions_norm.shape[1], activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

#### Compilado el Modelo

In [27]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

#### Entrenamiento del Modelo

In [35]:
model.fit(X_train, Y_train, validation_split=0.3, epochs=200, batch_size=50000, callbacks=[callback])

Epoch 1/200
17/17 [==============================] - 1s 32ms/step - loss: 7.3887e-05 - mean_squared_error: 7.3887e-05 - val_loss: 7.3253e-05 - val_mean_squared_error: 7.3253e-05
Epoch 2/200
17/17 [==============================] - 1s 30ms/step - loss: 7.3574e-05 - mean_squared_error: 7.3574e-05 - val_loss: 7.3025e-05 - val_mean_squared_error: 7.3025e-05
Epoch 3/200
17/17 [==============================] - 0s 24ms/step - loss: 7.3291e-05 - mean_squared_error: 7.3291e-05 - val_loss: 7.2851e-05 - val_mean_squared_error: 7.2851e-05
Epoch 4/200
17/17 [==============================] - 0s 23ms/step - loss: 7.3046e-05 - mean_squared_error: 7.3046e-05 - val_loss: 7.2471e-05 - val_mean_squared_error: 7.2471e-05
Epoch 5/200
17/17 [==============================] - 0s 23ms/step - loss: 7.2804e-05 - mean_squared_error: 7.2804e-05 - val_loss: 7.2278e-05 - val_mean_squared_error: 7.2278e-05
Epoch 6/200
17/17 [==============================] - 0s 24ms/step - loss: 7.2564e-05 - mean_squared_error: 7.2

Epoch 47/200
17/17 [==============================] - 0s 29ms/step - loss: 6.4157e-05 - mean_squared_error: 6.4157e-05 - val_loss: 6.3578e-05 - val_mean_squared_error: 6.3578e-05
Epoch 48/200
17/17 [==============================] - 0s 26ms/step - loss: 6.3990e-05 - mean_squared_error: 6.3990e-05 - val_loss: 6.3422e-05 - val_mean_squared_error: 6.3422e-05
Epoch 49/200
17/17 [==============================] - 0s 23ms/step - loss: 6.3900e-05 - mean_squared_error: 6.3900e-05 - val_loss: 6.3478e-05 - val_mean_squared_error: 6.3478e-05
Epoch 50/200
17/17 [==============================] - 0s 23ms/step - loss: 6.3778e-05 - mean_squared_error: 6.3778e-05 - val_loss: 6.3149e-05 - val_mean_squared_error: 6.3149e-05
Epoch 51/200
17/17 [==============================] - 0s 26ms/step - loss: 6.3558e-05 - mean_squared_error: 6.3558e-05 - val_loss: 6.3005e-05 - val_mean_squared_error: 6.3005e-05
Epoch 52/200
17/17 [==============================] - 0s 26ms/step - loss: 6.3439e-05 - mean_squared_erro

Epoch 93/200
17/17 [==============================] - 0s 27ms/step - loss: 5.8676e-05 - mean_squared_error: 5.8676e-05 - val_loss: 5.8363e-05 - val_mean_squared_error: 5.8363e-05
Epoch 94/200
17/17 [==============================] - 0s 25ms/step - loss: 5.8567e-05 - mean_squared_error: 5.8567e-05 - val_loss: 5.8281e-05 - val_mean_squared_error: 5.8281e-05
Epoch 95/200
17/17 [==============================] - 0s 24ms/step - loss: 5.8473e-05 - mean_squared_error: 5.8473e-05 - val_loss: 5.8201e-05 - val_mean_squared_error: 5.8201e-05
Epoch 96/200
17/17 [==============================] - 0s 25ms/step - loss: 5.8391e-05 - mean_squared_error: 5.8391e-05 - val_loss: 5.8089e-05 - val_mean_squared_error: 5.8089e-05
Epoch 97/200
17/17 [==============================] - 0s 26ms/step - loss: 5.8294e-05 - mean_squared_error: 5.8294e-05 - val_loss: 5.8030e-05 - val_mean_squared_error: 5.8030e-05
Epoch 98/200
17/17 [==============================] - 0s 25ms/step - loss: 5.8199e-05 - mean_squared_erro

17/17 [==============================] - 0s 25ms/step - loss: 5.4651e-05 - mean_squared_error: 5.4651e-05 - val_loss: 5.4689e-05 - val_mean_squared_error: 5.4689e-05
Epoch 139/200
17/17 [==============================] - 0s 26ms/step - loss: 5.4615e-05 - mean_squared_error: 5.4615e-05 - val_loss: 5.4615e-05 - val_mean_squared_error: 5.4615e-05
Epoch 140/200
17/17 [==============================] - 0s 26ms/step - loss: 5.4479e-05 - mean_squared_error: 5.4479e-05 - val_loss: 5.4668e-05 - val_mean_squared_error: 5.4668e-05
Epoch 141/200
17/17 [==============================] - 0s 23ms/step - loss: 5.4479e-05 - mean_squared_error: 5.4479e-05 - val_loss: 5.4417e-05 - val_mean_squared_error: 5.4417e-05
Epoch 142/200
17/17 [==============================] - 0s 23ms/step - loss: 5.4371e-05 - mean_squared_error: 5.4371e-05 - val_loss: 5.4513e-05 - val_mean_squared_error: 5.4513e-05
Epoch 143/200
17/17 [==============================] - 0s 23ms/step - loss: 5.4335e-05 - mean_squared_error: 5.433

17/17 [==============================] - 0s 23ms/step - loss: 5.1974e-05 - mean_squared_error: 5.1974e-05 - val_loss: 5.2204e-05 - val_mean_squared_error: 5.2204e-05
Epoch 184/200
17/17 [==============================] - 0s 22ms/step - loss: 5.1932e-05 - mean_squared_error: 5.1932e-05 - val_loss: 5.2284e-05 - val_mean_squared_error: 5.2284e-05
Epoch 185/200
17/17 [==============================] - 0s 23ms/step - loss: 5.1891e-05 - mean_squared_error: 5.1891e-05 - val_loss: 5.2246e-05 - val_mean_squared_error: 5.2246e-05
Epoch 186/200
17/17 [==============================] - 0s 23ms/step - loss: 5.1865e-05 - mean_squared_error: 5.1865e-05 - val_loss: 5.2211e-05 - val_mean_squared_error: 5.2211e-05


#### Evaluación del Modelo

In [36]:
loss, mean_sq = model.evaluate(X_test, Y_test)
print(f'mean_sq: {mean_sq} -> {mean_sq*K["K"]} RPM')
print(f'loss: {loss} -> {loss*K["K"]} RPM')

9000/9000 [==============================] - 8s 869us/step - loss: 5.0258e-05 - mean_squared_error: 5.0258e-051s - loss:
mean_sq: 5.0257789553143084e-05 -> 1.088907758819223 RPM
loss: 5.0257789553143084e-05 -> 1.088907758819223 RPM


#### Se guarda el Modelo

In [37]:
model.save(f'../Models/{dataset_name}.h5')

In [92]:
model = tf.keras.models.load_model(f'../Models/{dataset_name}.h5')
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 36)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                370       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 44        
Total params: 524
Trainable params: 524
Non-trainable params: 0
_________________________________________________________________


In [95]:
%%time
x_test = X_test.sample(n=3, random_state=1)
for index, sample in x_test.iterrows():
    print(model.predict([list(sample)])*K['K'])

[[14724.528 14720.853 14736.719 14730.789]]
[[14476.222 14475.466 14477.044 14474.773]]
[[14023.084 14020.965 14012.791 14017.1  ]]
Wall time: 225 ms


In [98]:
%%time
x_test = [0]*36
x_test[0] = 1
x_test[1] = -9.8
x_test[2] = 3
print(model.predict([list(x_test)])*K['K'])

[[12945.635 15027.058 13478.321 15521.83 ]]
Wall time: 79.1 ms
